# GEH Pharmacy Department

In [111]:
import pandas as pd
import numpy as np
import glob
import os
from datetime import *

## Data Cleaning

### Merge April & May Data

In [112]:
files_xls_1 = ['../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT (April 2021).xlsx', '../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT 2021 May.xlsx']

In [113]:
df = pd.DataFrame()

In [114]:
for f in files_xls_1:
    data = pd.read_excel(f, 'Template')
    df = df.append(data)

In [115]:
df

,Verifier/Typist,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Checker,Unnamed: 8,Unnamed: 9,...,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,Date,Case No,Room No,Ward,SCM / Handwritten TTO,No. of items,Time in / Verified,Time checked,Checked by,Time to meet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-04-01 00:00:00,1021023186,957,9W,SCM,2,08:59:00,09:04:00,LET,09:44:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-04-01 00:00:00,1021023191,509,5E,SCM,3,09:05:00,09:26:00,LET,09:50:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-04-01 00:00:00,1021023190,508,5E,SCM,4,09:07:00,09:19:00,LET,09:52:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-04-01 00:00:00,1021023367,932,9E,SCM,2,09:11:00,09:22:00,ELISE,09:56:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
df.columns = df.iloc[0]

In [117]:
df = df.drop(0)
df

,Date,Case No,Room No,Ward,SCM / Handwritten TTO,No. of items,Time in / Verified,Time checked,Checked by,Time to meet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-04-01 00:00:00,1021023186,957,9W,SCM,2,08:59:00,09:04:00,LET,09:44:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-04-01 00:00:00,1021023191,509,5E,SCM,3,09:05:00,09:26:00,LET,09:50:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-04-01 00:00:00,1021023190,508,5E,SCM,4,09:07:00,09:19:00,LET,09:52:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-04-01 00:00:00,1021023367,932,9E,SCM,2,09:11:00,09:22:00,ELISE,09:56:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2021-04-01 00:00:00,1021023455,909,9E,SCM,5,09:36:00,10:02:00,ELISE,10:21:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
df = df.dropna(axis=1, how='all')
df

,Date,Case No,Room No,Ward,SCM / Handwritten TTO,No. of items,Time in / Verified,Time checked,Checked by,Time to meet,Time reached,Time Counselled,Counselled by,Remarks,Limo Service,Timings (Checked),Timings (Reached),NaN
1,2021-04-01 00:00:00,1021023186,957,9W,SCM,2,08:59:00,09:04:00,LET,09:44:00,09:29:00,09:35:00,LET,NaN,NaN,00:05:00,00:30:00,NaN
2,2021-04-01 00:00:00,1021023191,509,5E,SCM,3,09:05:00,09:26:00,LET,09:50:00,09:29:00,09:32:00,ELISE,NaN,NaN,00:21:00,00:24:00,NaN
3,2021-04-01 00:00:00,1021023190,508,5E,SCM,4,09:07:00,09:19:00,LET,09:52:00,09:32:00,10:17:00,ELISE,P,NaN,00:12:00,00:25:00,NaN
4,2021-04-01 00:00:00,1021023367,932,9E,SCM,2,09:11:00,09:22:00,ELISE,09:56:00,09:37:00,09:40:00,LET,NaN,NaN,00:11:00,00:26:00,NaN
5,2021-04-01 00:00:00,1021023455,909,9E,SCM,5,09:36:00,10:02:00,ELISE,10:21:00,10:21:00,10:25:00,ELISE,NaN,NaN,00:26:00,00:45:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [119]:
df = df[pd.notnull(df['Case No'])]

In [120]:
df

,Date,Case No,Room No,Ward,SCM / Handwritten TTO,No. of items,Time in / Verified,Time checked,Checked by,Time to meet,Time reached,Time Counselled,Counselled by,Remarks,Limo Service,Timings (Checked),Timings (Reached),NaN
1,2021-04-01 00:00:00,1021023186,957,9W,SCM,2,08:59:00,09:04:00,LET,09:44:00,09:29:00,09:35:00,LET,NaN,NaN,00:05:00,00:30:00,NaN
2,2021-04-01 00:00:00,1021023191,509,5E,SCM,3,09:05:00,09:26:00,LET,09:50:00,09:29:00,09:32:00,ELISE,NaN,NaN,00:21:00,00:24:00,NaN
3,2021-04-01 00:00:00,1021023190,508,5E,SCM,4,09:07:00,09:19:00,LET,09:52:00,09:32:00,10:17:00,ELISE,P,NaN,00:12:00,00:25:00,NaN
4,2021-04-01 00:00:00,1021023367,932,9E,SCM,2,09:11:00,09:22:00,ELISE,09:56:00,09:37:00,09:40:00,LET,NaN,NaN,00:11:00,00:26:00,NaN
5,2021-04-01 00:00:00,1021023455,909,9E,SCM,5,09:36:00,10:02:00,ELISE,10:21:00,10:21:00,10:25:00,ELISE,NaN,NaN,00:26:00,00:45:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,NaN,1021037723,112,10W,SCM,2,09:24:00,10:53:00,TEL,10:09:00,11:10:00,11:17:00,TEL,patient shower,NaN,01:29:00,01:46:00,NaN
204,NaN,1021037197,921,9E,SCM,1,10:38:00,10:53:00,TEL,11:23:00,11:15:00,NaN,TEL,Patient not available,NaN,00:15:00,00:37:00,NaN
205,NaN,1021037427,101,10W,SCM,1,12:00:00,12:05:00,TEL,12:45:00,12:10:00,12:20:00,TEL,NaN,NaN,00:05:00,00:10:00,NaN
206,NaN,1021039189,928,9E,scm,3,12:57:00,13:22:00,WQ,13:42:00,13:40:00,13:47:00,TEL,NaN,NaN,00:25:00,00:43:00,NaN


In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497 entries, 1 to 207
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Date                    424 non-null    object
 1   Case No                 497 non-null    object
 2   Room No                 497 non-null    object
 3   Ward                    497 non-null    object
 4   SCM / Handwritten TTO   497 non-null    object
 5   No. of items            497 non-null    object
 6   Time in / Verified      491 non-null    object
 7   Time checked            471 non-null    object
 8   Checked by              471 non-null    object
 9   Time to meet            497 non-null    object
 10  Time reached            434 non-null    object
 11   Time Counselled        390 non-null    object
 12  Counselled by           423 non-null    object
 13  Remarks                 154 non-null    object
 14  Limo Service            1 non-null      object
 15  Timing

#### Drop columns

In [122]:
df2= df.copy()

In [123]:
df2 = df2.drop(df.iloc[:, 17:], axis=1)

In [124]:
#reset index 
df2 = df2.reset_index(drop=True)

#### Rename common columns

In [125]:
df2.rename({'Date ': 'Date', 
            'Case No': 'Case Number', 
            'Room No': 'Room',
            'SCM / Handwritten TTO ': 'TTO Type', 
            'No. of items':'No. of Drugs', 
            'Time in / Verified': 'DateTime TTO Received', # the start time
            'Time checked':'DateTime TTO Checked', 
            'Time reached ': 'DateTime TTO Dispensed', # the end time 
            ' Time Counselled':'DateTime (Counselled)', # end time if patients need to be counselled
           'Limo Service ':'Limo Service'},  
           axis=1, inplace=True)

In [126]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Date                    424 non-null    object
 1   Case Number             497 non-null    object
 2   Room                    497 non-null    object
 3   Ward                    497 non-null    object
 4   TTO Type                497 non-null    object
 5   No. of Drugs            497 non-null    object
 6   DateTime TTO Received   491 non-null    object
 7   DateTime TTO Checked    471 non-null    object
 8   Checked by              471 non-null    object
 9   Time to meet            497 non-null    object
 10  DateTime TTO Dispensed  434 non-null    object
 11  DateTime (Counselled)   390 non-null    object
 12  Counselled by           423 non-null    object
 13  Remarks                 154 non-null    object
 14  Limo Service            1 non-null      object
 15  Timing

#### Handle missing value

In [127]:
list(df2.columns)

['Date',
 'Case Number',
 'Room',
 'Ward',
 'TTO Type',
 'No. of Drugs',
 'DateTime TTO Received',
 'DateTime TTO Checked',
 'Checked by',
 'Time to meet',
 'DateTime TTO Dispensed',
 'DateTime (Counselled)',
 'Counselled by',
 'Remarks',
 'Limo Service',
 'Timings (Checked)',
 'Timings (Reached)']

In [128]:
df2.isnull().sum()

0
Date                       73
Case Number                 0
Room                        0
Ward                        0
TTO Type                    0
No. of Drugs                0
DateTime TTO Received       6
DateTime TTO Checked       26
Checked by                 26
Time to meet                0
DateTime TTO Dispensed     63
DateTime (Counselled)     107
Counselled by              74
Remarks                   343
Limo Service              496
Timings (Checked)           8
Timings (Reached)          11
dtype: int64

##### Missing date

In [129]:
# change to datetime formate
df2['Date'] = pd.to_datetime(df2['Date'])

In [130]:
missing_date = df2[df2['Date'].isnull()]
missing_date

,Date,Case Number,Room,Ward,TTO Type,No. of Drugs,DateTime TTO Received,DateTime TTO Checked,Checked by,Time to meet,DateTime TTO Dispensed,DateTime (Counselled),Counselled by,Remarks,Limo Service,Timings (Checked),Timings (Reached)
386,NaT,1021035404,609,6E,SCM,3,09:10:00,09:22:00,HEMIN,09:55:00,09:52:00,09:55:00,HEMIN,NaN,NaN,00:12:00,00:42:00
387,NaT,1021033687,919,9E,SCM,6,09:21:00,09:39:00,HEMIN,10:06:00,11:02:00,11:09:00,LJY,NaN,NaN,00:18:00,01:41:00
388,NaT,1021032404,228,10W,SCM,3,09:33:00,10:10:00,LJY,10:18:00,10:40:00,10:44:00,HEMIN,NaN,NaN,00:37:00,01:07:00
389,NaT,1021034765,916,9E,SCM,4,09:35:00,09:43:00,HEMIN,10:20:00,10:00:00,10:04:00,AKH,NaN,NaN,00:08:00,00:25:00
390,NaT,1021035355,755,7W,SCM,2,09:41:00,09:51:00,LJY,10:26:00,NaN,NaN,NaN,WAIT FOR SON,NaN,00:10:00,1899-12-30 14:19:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,NaT,1021037723,112,10W,SCM,2,09:24:00,10:53:00,TEL,10:09:00,11:10:00,11:17:00,TEL,patient shower,NaN,01:29:00,01:46:00
493,NaT,1021037197,921,9E,SCM,1,10:38:00,10:53:00,TEL,11:23:00,11:15:00,NaN,TEL,Patient not available,NaN,00:15:00,00:37:00
494,NaT,1021037427,101,10W,SCM,1,12:00:00,12:05:00,TEL,12:45:00,12:10:00,12:20:00,TEL,NaN,NaN,00:05:00,00:10:00
495,NaT,1021039189,928,9E,scm,3,12:57:00,13:22:00,WQ,13:42:00,13:40:00,13:47:00,TEL,NaN,NaN,00:25:00,00:43:00


In [131]:
# fill up missing date based on previous known value 
df2['Date'] = df2['Date'].fillna(method='ffill')
df2.tail()

,Date,Case Number,Room,Ward,TTO Type,No. of Drugs,DateTime TTO Received,DateTime TTO Checked,Checked by,Time to meet,DateTime TTO Dispensed,DateTime (Counselled),Counselled by,Remarks,Limo Service,Timings (Checked),Timings (Reached)
492,2021-05-31,1021037723,112,10W,SCM,2,09:24:00,10:53:00,TEL,10:09:00,11:10:00,11:17:00,TEL,patient shower,NaN,01:29:00,01:46:00
493,2021-05-31,1021037197,921,9E,SCM,1,10:38:00,10:53:00,TEL,11:23:00,11:15:00,NaN,TEL,Patient not available,NaN,00:15:00,00:37:00
494,2021-05-31,1021037427,101,10W,SCM,1,12:00:00,12:05:00,TEL,12:45:00,12:10:00,12:20:00,TEL,NaN,NaN,00:05:00,00:10:00
495,2021-05-31,1021039189,928,9E,scm,3,12:57:00,13:22:00,WQ,13:42:00,13:40:00,13:47:00,TEL,NaN,NaN,00:25:00,00:43:00
496,2021-05-31,1021038490,622,TU,SCM,13,11:05:00,13:15:00,WQ,11:50:00,13:25:00,13:39:00,TEL,waiting for clinic TTO,NaN,02:10:00,02:20:00


##### Missing Time TTO Received

In [132]:
# drop the one missing row in april dataset bc no time in cant calculate time taken
# drop row in may too for same reason
df2 = df2.dropna(subset=['DateTime TTO Received'])

In [133]:
df2.isnull().sum()

0
Date                        0
Case Number                 0
Room                        0
Ward                        0
TTO Type                    0
No. of Drugs                0
DateTime TTO Received       0
DateTime TTO Checked       24
Checked by                 25
Time to meet                0
DateTime TTO Dispensed     60
DateTime (Counselled)     104
Counselled by              71
Remarks                   340
Limo Service              490
Timings (Checked)           8
Timings (Reached)          11
dtype: int64

#### EDA

In [134]:
df2

,Date,Case Number,Room,Ward,TTO Type,No. of Drugs,DateTime TTO Received,DateTime TTO Checked,Checked by,Time to meet,DateTime TTO Dispensed,DateTime (Counselled),Counselled by,Remarks,Limo Service,Timings (Checked),Timings (Reached)
0,2021-04-01,1021023186,957,9W,SCM,2,08:59:00,09:04:00,LET,09:44:00,09:29:00,09:35:00,LET,NaN,NaN,00:05:00,00:30:00
1,2021-04-01,1021023191,509,5E,SCM,3,09:05:00,09:26:00,LET,09:50:00,09:29:00,09:32:00,ELISE,NaN,NaN,00:21:00,00:24:00
2,2021-04-01,1021023190,508,5E,SCM,4,09:07:00,09:19:00,LET,09:52:00,09:32:00,10:17:00,ELISE,P,NaN,00:12:00,00:25:00
3,2021-04-01,1021023367,932,9E,SCM,2,09:11:00,09:22:00,ELISE,09:56:00,09:37:00,09:40:00,LET,NaN,NaN,00:11:00,00:26:00
4,2021-04-01,1021023455,909,9E,SCM,5,09:36:00,10:02:00,ELISE,10:21:00,10:21:00,10:25:00,ELISE,NaN,NaN,00:26:00,00:45:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,2021-05-31,1021037723,112,10W,SCM,2,09:24:00,10:53:00,TEL,10:09:00,11:10:00,11:17:00,TEL,patient shower,NaN,01:29:00,01:46:00
493,2021-05-31,1021037197,921,9E,SCM,1,10:38:00,10:53:00,TEL,11:23:00,11:15:00,NaN,TEL,Patient not available,NaN,00:15:00,00:37:00
494,2021-05-31,1021037427,101,10W,SCM,1,12:00:00,12:05:00,TEL,12:45:00,12:10:00,12:20:00,TEL,NaN,NaN,00:05:00,00:10:00
495,2021-05-31,1021039189,928,9E,scm,3,12:57:00,13:22:00,WQ,13:42:00,13:40:00,13:47:00,TEL,NaN,NaN,00:25:00,00:43:00


##### Day of Week

In [135]:
df2['Weekday'] = df2['Date'].dt.day_name()

In [136]:
# handle Unknown string format: 9.:09 in Time TTO reached
df2['DateTime TTO Dispensed'] = df2['DateTime TTO Dispensed'].replace('9.:09', 'NA')

In [137]:
#tto_received = pd.to_datetime(df2['Time TTO Received'].astype(str)) 
#tto_reached = pd.to_datetime(df2[df2['Time TTO Reached'] != 'NA']['Time TTO Reached'].astype(str))
#tto_counselled = pd.to_datetime(df2[df2['Time TTO Counselled'] != 'NA']['Time TTO Counselled'].astype(str))

#df2['Overall Time Taken (mins)'] = tto_reached.sub(tto_received).dt.total_seconds().div(60)

In [138]:
#df2.info()

In [139]:
#df2['time_diff'] = pd.Timestamp(df2['Time TTO Received']) - pd.Timestamp(df['end_time'])   
#def f(row):
#    if row['Time TTO Counselled'] != " ":
#        val = tto_counselled.sub(tto_received).dt.total_seconds().div(60)
    #elif row['Time TTO Counselled'] <  row['Time TTO Reached']:
    #    val = tto_reached.sub(tto_received).dt.total_seconds().div(60)
#    else:
#        val = tto_reached.sub(tto_received).dt.total_seconds().div(60)
#    return val

In [140]:
#df2['Overall Time Taken (mins)_2'] = df2.apply(f, axis=1)
#df2

#### Export to CSV for Final Merge

In [141]:
df2.to_csv("../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT (April-May 2021).csv", index = False, header=True)

### Merge remaining months

In [142]:
files_xls_2 = ['../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT 2021 June.xlsx',
 '../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT 2021 July.xlsx',
 '../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT 2021 Aug.xlsx',
'../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT 2021 Sep.xlsx',
 '../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT 2021 Nov.xlsx', 
'../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT 2021 Oct.xlsx', 
 '../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT 2021 Dec.xlsx']

In [143]:
df3 = pd.DataFrame()
for f in files_xls_2:
    data = pd.read_excel(f)
    df3 = df3.append(data)
df3

,Date,Case Number,Room,TTO Type,No. of Items,Time In,Time Checked,Pharmacist_Checked,Time Reached,Time Dispensed,...,status_TTOReached,status_Dispensed,status_TTOIn,Title,TTOaddon,Item Type,Path,Time,Unnamed: 23,Unnamed: 24
0,2021-06-01 01:00:00,1.021039e+09,953,SCM,3.0,2021-06-01 09:13:00,2021-06-01 09:07:17,Ray Hng Yeo,2021-06-01 09:28:26,2021-06-01 09:31:37,...,True,True,True,NaN,NaN,Item,personal/rayhng_yeo_gleneagles_com_sg/Lists/TT...,NaN,NaN,NaN
1,2021-06-01 01:00:00,1.021039e+09,623,SCM,1.0,2021-06-01 09:30:00,2021-06-01 09:38:25,Ray Hng Yeo,2021-06-01 10:07:32,2021-06-01 10:09:24,...,True,True,True,NaN,NaN,Item,personal/rayhng_yeo_gleneagles_com_sg/Lists/TT...,NaN,NaN,NaN
2,2021-06-01 01:00:00,1.021039e+09,757,SCM,1.0,2021-06-01 09:36:00,2021-06-01 09:43:48,Nur Shahida Binte Ahmad,2021-06-01 10:30:09,2021-06-01 10:30:18,...,True,True,True,NaN,NaN,Item,personal/rayhng_yeo_gleneagles_com_sg/Lists/TT...,NaN,NaN,NaN
3,2021-06-01 01:00:00,1.021038e+09,916,SCM,4.0,2021-06-01 09:46:00,2021-06-01 09:49:54,Nur Shahida Binte Ahmad,2021-06-01 10:12:22,2021-06-01 10:13:34,...,True,True,True,NaN,NaN,Item,personal/rayhng_yeo_gleneagles_com_sg/Lists/TT...,NaN,NaN,NaN
4,2021-06-01 01:00:00,1.021039e+09,958,NaN,NaN,2021-06-01 08:55:00,2021-06-01 09:56:00,Wan Qing,2021-06-01 10:18:00,NaT,...,True,False,True,NaN,NaN,Item,personal/rayhng_yeo_gleneagles_com_sg/Lists/TT...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,2021-12-01 00:00:00,1.021086e+09,905,SCM,9.0,2021-12-01 09:39:00,2021-12-01 10:30:00,Hemin Chang,2021-12-01 10:44:00,2021-12-01 10:50:00,...,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
325,2021-12-01 00:00:00,1.021087e+09,102,SCM,1.0,2021-12-01 11:30:00,2021-12-01 11:42:00,Zhao Wan Qing,2021-12-01 12:08:00,2021-12-01 12:12:00,...,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
326,2021-12-01 00:00:00,1.021085e+09,113,SCM,1.0,2021-12-01 12:26:00,2021-12-01 12:38:00,Zhao Wan Qing,2021-12-01 12:45:00,NaT,...,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
327,2021-12-01 00:00:00,1.021085e+09,113,SCM,1.0,2021-12-01 13:11:00,2021-12-01 13:17:00,Yi Zhen Tay,2021-12-01 13:34:00,2021-12-01 13:37:00,...,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [144]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2589 entries, 0 to 328
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  2312 non-null   datetime64[ns]
 1   Case Number           2312 non-null   float64       
 2   Room                  2312 non-null   object        
 3   TTO Type              2303 non-null   object        
 4   No. of Items          2309 non-null   float64       
 5   Time In               2283 non-null   datetime64[ns]
 6   Time Checked          2114 non-null   datetime64[ns]
 7   Pharmacist_Checked    2105 non-null   object        
 8   Time Reached          1969 non-null   datetime64[ns]
 9   Time Dispensed        1668 non-null   datetime64[ns]
 10  Pharmacist_Dispensed  1831 non-null   object        
 11  Remarks               421 non-null    object        
 12  Dispatcher            1 non-null      object        
 13  TTO_TAT_Calc       

In [145]:
#reset index 
df3 = df3.reset_index(drop=True)

#### Drop columns

In [146]:
# drop columns that should be ignored
df3 = df3.drop(['TTO_TAT_Calc', 'Total number of TTO', 'Status_Time_Met', 'ID', 'Title', 'TTOaddon', 'Item Type', 'Path', 'Unnamed: 23', 'Unnamed: 24'], axis = 1)
# drop other redundant columns
#df3 = df3.drop(['Pharmacist_Checked', 'Pharmacist_Dispensed', 'Dispatcher' ], axis = 1)

#### Rename common columns

In [147]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2589 entries, 0 to 2588
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  2312 non-null   datetime64[ns]
 1   Case Number           2312 non-null   float64       
 2   Room                  2312 non-null   object        
 3   TTO Type              2303 non-null   object        
 4   No. of Items          2309 non-null   float64       
 5   Time In               2283 non-null   datetime64[ns]
 6   Time Checked          2114 non-null   datetime64[ns]
 7   Pharmacist_Checked    2105 non-null   object        
 8   Time Reached          1969 non-null   datetime64[ns]
 9   Time Dispensed        1668 non-null   datetime64[ns]
 10  Pharmacist_Dispensed  1831 non-null   object        
 11  Remarks               421 non-null    object        
 12  Dispatcher            1 non-null      object        
 13  status_CheckedTTO 

In [148]:
df3.rename({'No. of Items':'No. of Drugs', 
            'Time In': 'DateTime TTO Received', # the start time
            'Time Checked':'DateTime TTO Checked', 
            'Time Reached': 'DateTime TTO Dispensed'}, # the end time  'Time Dispensed' : 'DateTime TTO Dispensed'},
           axis=1, inplace=True)

In [149]:
columns = ['DateTime TTO Received', 'DateTime TTO Checked', 'DateTime TTO Dispensed']
           #, 'DateTime TTO Counselled']

for column in columns:
    # remove date in the time metric
    #df3[column]=df3[column].dt.time 
    df3[column] = df3[column].astype(str).str.split(' ').str[-1]

In [150]:
df3

,Date,Case Number,Room,TTO Type,No. of Drugs,DateTime TTO Received,DateTime TTO Checked,Pharmacist_Checked,DateTime TTO Dispensed,Time Dispensed,Pharmacist_Dispensed,Remarks,Dispatcher,status_CheckedTTO,status_TTOReached,status_Dispensed,status_TTOIn,Time
0,2021-06-01 01:00:00,1.021039e+09,953,SCM,3.0,09:13:00,09:07:17,Ray Hng Yeo,09:28:26,2021-06-01 09:31:37,Zhao Wan Qing,NaN,NaN,True,True,True,True,NaN
1,2021-06-01 01:00:00,1.021039e+09,623,SCM,1.0,09:30:00,09:38:25,Ray Hng Yeo,10:07:32,2021-06-01 10:09:24,Ray Hng Yeo,NaN,NaN,True,True,True,True,NaN
2,2021-06-01 01:00:00,1.021039e+09,757,SCM,1.0,09:36:00,09:43:48,Nur Shahida Binte Ahmad,10:30:09,2021-06-01 10:30:18,Ray Hng Yeo,NaN,NaN,True,True,True,True,NaN
3,2021-06-01 01:00:00,1.021038e+09,916,SCM,4.0,09:46:00,09:49:54,Nur Shahida Binte Ahmad,10:12:22,2021-06-01 10:13:34,Zhao Wan Qing,NaN,NaN,True,True,True,True,NaN
4,2021-06-01 01:00:00,1.021039e+09,958,NaN,NaN,08:55:00,09:56:00,Wan Qing,10:18:00,NaT,NaN,9W remaining meds,NaN,True,True,False,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2584,2021-12-01 00:00:00,1.021086e+09,905,SCM,9.0,09:39:00,10:30:00,Hemin Chang,10:44:00,2021-12-01 10:50:00,Hemin Chang,"[""9W remaining meds""]",NaN,Yes,Yes,Yes,Yes,NaN
2585,2021-12-01 00:00:00,1.021087e+09,102,SCM,1.0,11:30:00,11:42:00,Zhao Wan Qing,12:08:00,2021-12-01 12:12:00,Zhao Wan Qing,NaN,NaN,Yes,Yes,Yes,Yes,NaN
2586,2021-12-01 00:00:00,1.021085e+09,113,SCM,1.0,12:26:00,12:38:00,Zhao Wan Qing,12:45:00,NaT,Zhao Wan Qing,NaN,NaN,Yes,Yes,Yes,Yes,NaN
2587,2021-12-01 00:00:00,1.021085e+09,113,SCM,1.0,13:11:00,13:17:00,Yi Zhen Tay,13:34:00,2021-12-01 13:37:00,Hemin Chang,NaN,NaN,Yes,Yes,Yes,Yes,NaN


In [151]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2589 entries, 0 to 2588
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    2312 non-null   datetime64[ns]
 1   Case Number             2312 non-null   float64       
 2   Room                    2312 non-null   object        
 3   TTO Type                2303 non-null   object        
 4   No. of Drugs            2309 non-null   float64       
 5   DateTime TTO Received   2589 non-null   object        
 6   DateTime TTO Checked    2589 non-null   object        
 7   Pharmacist_Checked      2105 non-null   object        
 8   DateTime TTO Dispensed  2589 non-null   object        
 9   Time Dispensed          1668 non-null   datetime64[ns]
 10  Pharmacist_Dispensed    1831 non-null   object        
 11  Remarks                 421 non-null    object        
 12  Dispatcher              1 non-null      object  

#### Handle missing value

In [152]:
df3.isnull().sum()

Date                       277
Case Number                277
Room                       277
TTO Type                   286
No. of Drugs               280
DateTime TTO Received        0
DateTime TTO Checked         0
Pharmacist_Checked         484
DateTime TTO Dispensed       0
Time Dispensed             921
Pharmacist_Dispensed       758
Remarks                   2168
Dispatcher                2588
status_CheckedTTO          277
status_TTOReached          277
status_Dispensed           277
status_TTOIn               277
Time                       657
dtype: int64

##### Missing Date

In [153]:
# change to datetime format
df3['Date'] = pd.to_datetime(df3['Date'])

In [154]:
missing_date = df3[df3['Date'].isnull()]
missing_date

,Date,Case Number,Room,TTO Type,No. of Drugs,DateTime TTO Received,DateTime TTO Checked,Pharmacist_Checked,DateTime TTO Dispensed,Time Dispensed,Pharmacist_Dispensed,Remarks,Dispatcher,status_CheckedTTO,status_TTOReached,status_Dispensed,status_TTOIn,Time
1061,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1062,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1063,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1064,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1065,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2255,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2256,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2257,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2258,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [155]:
#drop rows
df3 = df3.dropna(subset=['Date'])

In [156]:
# missing pharmacist dispensed
missing_date = df3[df3['Pharmacist_Dispensed'].isnull()]
missing_date.head()

,Date,Case Number,Room,TTO Type,No. of Drugs,DateTime TTO Received,DateTime TTO Checked,Pharmacist_Checked,DateTime TTO Dispensed,Time Dispensed,Pharmacist_Dispensed,Remarks,Dispatcher,status_CheckedTTO,status_TTOReached,status_Dispensed,status_TTOIn,Time
4,2021-06-01 01:00:00,1.021039e+09,958,NaN,NaN,08:55:00,09:56:00,Wan Qing,10:18:00,NaT,NaN,9W remaining meds,NaN,True,True,False,True,NaN
6,2021-06-01 01:00:00,1.021040e+09,936,SCM,2.0,01:00:00,10:49:00,NaN,10:58:00,NaT,NaN,Passed to SN,NaN,True,True,False,True,NaN
17,2021-06-02 01:00:00,1.021039e+09,948,Handwritten,2.0,01:00:00,09:13:00,NaN,10:51:00,NaT,NaN,Passed to SN,NaN,True,True,False,True,NaN
18,2021-06-02 01:00:00,1.021024e+09,932,SCM,5.0,10:40:00,10:46:00,Ray Hng Yeo,10:55:00,NaT,NaN,Passed to SN,NaN,True,True,False,True,NaN
22,2021-06-02 01:00:00,1.021040e+09,963,SCM,2.0,10:43:00,11:02:00,Kwok Fai,NaT,NaT,NaN,NaN,NaN,True,False,False,True,NaN


In [157]:
df3['No. of Drugs'].replace(np.nan, 'NA', inplace = True)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [158]:
missing_date = df3[df3['Time'].notnull()]
missing_date

,Date,Case Number,Room,TTO Type,No. of Drugs,DateTime TTO Received,DateTime TTO Checked,Pharmacist_Checked,DateTime TTO Dispensed,Time Dispensed,Pharmacist_Dispensed,Remarks,Dispatcher,status_CheckedTTO,status_TTOReached,status_Dispensed,status_TTOIn,Time
328,2021-07-01,1.021046e+09,226,SCM,5,08:28:00,NaT,NaN,NaT,NaT,NaN,NaN,NaN,False,False,False,True,493.888889
329,2021-07-01,1.021047e+09,621,SCM,1,08:57:00,09:02:00,Yi Zhen Tay,09:40:00,2021-07-01 09:55:00,Yi Zhen Tay,Passed to SN,NaN,True,True,True,True,41.805556
330,2021-07-01,1.021047e+09,622,SCM,5,09:14:00,09:29:00,Yi Zhen Tay,09:42:00,2021-07-01 09:52:00,Yi Zhen Tay,NaN,NaN,True,True,True,True,27.222222
331,2021-07-01,1.231045e+09,113,SCM,4,09:19:00,09:35:00,Yi Li Elise Yap,09:53:00,2021-07-01 10:28:00,Samuel Lai,Patient not available,NaN,True,True,True,True,33.055556
332,2021-07-01,1.021046e+09,507,SCM,2,09:21:00,09:41:00,Yi Li Elise Yap,09:59:00,NaT,Samuel Lai,NaN,NaN,True,True,True,True,36.944444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2202,2021-10-01,1.021069e+09,605,SCM,3,14:25:00,14:48:00,Yi Li Elise Yap,15:06:00,2021-10-01 15:09:00,Samuel,NaN,NaN,Yes,Yes,Yes,Yes,39.861111
2203,2021-10-01,1.021071e+09,512,SCM,2,16:36:00,16:58:00,Yi Li Elise Yap,17:02:00,NaT,Elise,"[""Passed to SN""]",NaN,Yes,Yes,No,Yes,25.277778
2204,2021-10-01,1.021070e+09,921,SCM,4,16:47:00,16:54:00,Yi Li Elise Yap,17:07:00,NaT,Elise,"[""Passed to SN""]",NaN,Yes,Yes,No,Yes,19.444444
2205,2021-10-01,1.021072e+09,510,SCM,3,18:49:00,18:57:00,Choon Yuen,18:59:00,2021-10-01 19:05:00,Yi Li Elise Yap,NaN,NaN,Yes,Yes,Yes,Yes,9.722222


In [159]:
df3.isnull().sum()

Date                         0
Case Number                  0
Room                         0
TTO Type                     9
No. of Drugs                 0
DateTime TTO Received        0
DateTime TTO Checked         0
Pharmacist_Checked         207
DateTime TTO Dispensed       0
Time Dispensed             644
Pharmacist_Dispensed       481
Remarks                   1891
Dispatcher                2311
status_CheckedTTO            0
status_TTOReached            0
status_Dispensed             0
status_TTOIn                 0
Time                       657
dtype: int64

#### EDA

##### Day of Week

In [160]:
# add weekday column
df3['Weekday'] = df3['Date'].dt.day_name()

<ipython-input-160-a1401e15acbe>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Weekday'] = df3['Date'].dt.day_name()


#### Export to CSV for Final Merge

In [161]:
df3.to_csv("../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT (June-Dec 2021).csv", index = None, header=True)

### Final Merge of the above CSVs

In [162]:
merged_df = pd.concat([df2, df3])

In [163]:
merged_df

,Date,Case Number,Room,Ward,TTO Type,No. of Drugs,DateTime TTO Received,DateTime TTO Checked,Checked by,Time to meet,...,Weekday,Pharmacist_Checked,Time Dispensed,Pharmacist_Dispensed,Dispatcher,status_CheckedTTO,status_TTOReached,status_Dispensed,status_TTOIn,Time
0,2021-04-01,1021023186,957,9W,SCM,2,08:59:00,09:04:00,LET,09:44:00,...,Thursday,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-04-01,1021023191,509,5E,SCM,3,09:05:00,09:26:00,LET,09:50:00,...,Thursday,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-04-01,1021023190,508,5E,SCM,4,09:07:00,09:19:00,LET,09:52:00,...,Thursday,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-04-01,1021023367,932,9E,SCM,2,09:11:00,09:22:00,ELISE,09:56:00,...,Thursday,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-04-01,1021023455,909,9E,SCM,5,09:36:00,10:02:00,ELISE,10:21:00,...,Thursday,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2584,2021-12-01,1.02109e+09,905,NaN,SCM,9,09:39:00,10:30:00,NaN,NaN,...,Wednesday,Hemin Chang,2021-12-01 10:50:00,Hemin Chang,NaN,Yes,Yes,Yes,Yes,NaN
2585,2021-12-01,1.02109e+09,102,NaN,SCM,1,11:30:00,11:42:00,NaN,NaN,...,Wednesday,Zhao Wan Qing,2021-12-01 12:12:00,Zhao Wan Qing,NaN,Yes,Yes,Yes,Yes,NaN
2586,2021-12-01,1.02108e+09,113,NaN,SCM,1,12:26:00,12:38:00,NaN,NaN,...,Wednesday,Zhao Wan Qing,NaT,Zhao Wan Qing,NaN,Yes,Yes,Yes,Yes,NaN
2587,2021-12-01,1.02108e+09,113,NaN,SCM,1,13:11:00,13:17:00,NaN,NaN,...,Wednesday,Yi Zhen Tay,2021-12-01 13:37:00,Hemin Chang,NaN,Yes,Yes,Yes,Yes,NaN


In [164]:
columns = ['DateTime TTO Received',
           'DateTime TTO Checked', 
           'DateTime TTO Dispensed']
for column in columns:
    merged_df.loc[:,column] = pd.to_datetime((merged_df['Date'].astype(str) + ' ' + merged_df[column].astype(str)), infer_datetime_format=True, errors='coerce')

/opt/anaconda3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname NA identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [165]:
merged_df['Date'] = pd.to_datetime(merged_df['Date']).dt.normalize()

In [166]:
merged_df.head()

,Date,Case Number,Room,Ward,TTO Type,No. of Drugs,DateTime TTO Received,DateTime TTO Checked,Checked by,Time to meet,...,Weekday,Pharmacist_Checked,Time Dispensed,Pharmacist_Dispensed,Dispatcher,status_CheckedTTO,status_TTOReached,status_Dispensed,status_TTOIn,Time
0,2021-04-01,1021023186,957,9W,SCM,2,2021-04-01 08:59:00,2021-04-01 09:04:00,LET,09:44:00,...,Thursday,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-04-01,1021023191,509,5E,SCM,3,2021-04-01 09:05:00,2021-04-01 09:26:00,LET,09:50:00,...,Thursday,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-04-01,1021023190,508,5E,SCM,4,2021-04-01 09:07:00,2021-04-01 09:19:00,LET,09:52:00,...,Thursday,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-04-01,1021023367,932,9E,SCM,2,2021-04-01 09:11:00,2021-04-01 09:22:00,ELISE,09:56:00,...,Thursday,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-04-01,1021023455,909,9E,SCM,5,2021-04-01 09:36:00,2021-04-01 10:02:00,ELISE,10:21:00,...,Thursday,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [167]:
list(merged_df.columns)

['Date',
 'Case Number',
 'Room',
 'Ward',
 'TTO Type',
 'No. of Drugs',
 'DateTime TTO Received',
 'DateTime TTO Checked',
 'Checked by',
 'Time to meet',
 'DateTime TTO Dispensed',
 'DateTime (Counselled)',
 'Counselled by',
 'Remarks',
 'Limo Service',
 'Timings (Checked)',
 'Timings (Reached)',
 'Weekday',
 'Pharmacist_Checked',
 'Time Dispensed',
 'Pharmacist_Dispensed',
 'Dispatcher',
 'status_CheckedTTO',
 'status_TTOReached',
 'status_Dispensed',
 'status_TTOIn',
 'Time']

In [168]:
merged_df = merged_df.drop(['Pharmacist_Checked','Pharmacist_Dispensed','Dispatcher',
                            'status_CheckedTTO','status_TTOReached','status_Dispensed','status_TTOIn','Time', 'Time Dispensed'],
                           axis=1)

In [169]:
merged_df.rename({'Timings (Checked)' : 'Time Taken (Received to Checked)',
              'Timings (Reached)': 'Time Taken (Received to Dispensed)'},  
           axis=1, inplace=True)
    

In [170]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2803 entries, 0 to 2588
Data columns (total 18 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Date                                2803 non-null   datetime64[ns]
 1   Case Number                         2803 non-null   object        
 2   Room                                2803 non-null   object        
 3   Ward                                491 non-null    object        
 4   TTO Type                            2794 non-null   object        
 5   No. of Drugs                        2803 non-null   object        
 6   DateTime TTO Received               2774 non-null   datetime64[ns]
 7   DateTime TTO Checked                2581 non-null   datetime64[ns]
 8   Checked by                          466 non-null    object        
 9   Time to meet                        491 non-null    object        
 10  DateTime TTO Dispensed  

In [171]:
merged_df[merged_df.index.duplicated()]

,Date,Case Number,Room,Ward,TTO Type,No. of Drugs,DateTime TTO Received,DateTime TTO Checked,Checked by,Time to meet,DateTime TTO Dispensed,DateTime (Counselled),Counselled by,Remarks,Limo Service,Time Taken (Received to Checked),Time Taken (Received to Dispensed),Weekday
0,2021-06-01,1.02104e+09,953,NaN,SCM,3,2021-06-01 09:13:00,2021-06-01 09:07:17,NaN,NaN,2021-06-01 09:28:26,NaN,NaN,NaN,NaN,NaN,NaN,Tuesday
1,2021-06-01,1.02104e+09,623,NaN,SCM,1,2021-06-01 09:30:00,2021-06-01 09:38:25,NaN,NaN,2021-06-01 10:07:32,NaN,NaN,NaN,NaN,NaN,NaN,Tuesday
2,2021-06-01,1.02104e+09,757,NaN,SCM,1,2021-06-01 09:36:00,2021-06-01 09:43:48,NaN,NaN,2021-06-01 10:30:09,NaN,NaN,NaN,NaN,NaN,NaN,Tuesday
3,2021-06-01,1.02104e+09,916,NaN,SCM,4,2021-06-01 09:46:00,2021-06-01 09:49:54,NaN,NaN,2021-06-01 10:12:22,NaN,NaN,NaN,NaN,NaN,NaN,Tuesday
4,2021-06-01,1.02104e+09,958,NaN,NaN,NA,2021-06-01 08:55:00,2021-06-01 09:56:00,NaN,NaN,2021-06-01 10:18:00,NaN,NaN,9W remaining meds,NaN,NaN,NaN,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,2021-07-14,1.02105e+09,916,NaN,SCM,2,2021-07-14 09:43:00,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday
493,2021-07-14,1.02105e+09,110,NaN,Handwritten,6,2021-07-14 09:14:00,2021-07-14 10:11:00,NaN,NaN,2021-07-14 12:35:00,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday
494,2021-07-14,1.02105e+09,226,NaN,SCM,1,2021-07-14 10:17:00,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday
495,2021-07-14,1.02105e+09,740,NaN,Handwritten,3,2021-07-14 09:28:00,2021-07-14 09:39:00,NaN,NaN,2021-07-14 10:44:00,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday


In [172]:
#reset index 
merged_df = merged_df.reset_index(drop=True)

In [173]:
merged_df

,Date,Case Number,Room,Ward,TTO Type,No. of Drugs,DateTime TTO Received,DateTime TTO Checked,Checked by,Time to meet,DateTime TTO Dispensed,DateTime (Counselled),Counselled by,Remarks,Limo Service,Time Taken (Received to Checked),Time Taken (Received to Dispensed),Weekday
0,2021-04-01,1021023186,957,9W,SCM,2,2021-04-01 08:59:00,2021-04-01 09:04:00,LET,09:44:00,2021-04-01 09:29:00,09:35:00,LET,NaN,NaN,00:05:00,00:30:00,Thursday
1,2021-04-01,1021023191,509,5E,SCM,3,2021-04-01 09:05:00,2021-04-01 09:26:00,LET,09:50:00,2021-04-01 09:29:00,09:32:00,ELISE,NaN,NaN,00:21:00,00:24:00,Thursday
2,2021-04-01,1021023190,508,5E,SCM,4,2021-04-01 09:07:00,2021-04-01 09:19:00,LET,09:52:00,2021-04-01 09:32:00,10:17:00,ELISE,P,NaN,00:12:00,00:25:00,Thursday
3,2021-04-01,1021023367,932,9E,SCM,2,2021-04-01 09:11:00,2021-04-01 09:22:00,ELISE,09:56:00,2021-04-01 09:37:00,09:40:00,LET,NaN,NaN,00:11:00,00:26:00,Thursday
4,2021-04-01,1021023455,909,9E,SCM,5,2021-04-01 09:36:00,2021-04-01 10:02:00,ELISE,10:21:00,2021-04-01 10:21:00,10:25:00,ELISE,NaN,NaN,00:26:00,00:45:00,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2798,2021-12-01,1.02109e+09,905,NaN,SCM,9,2021-12-01 09:39:00,2021-12-01 10:30:00,NaN,NaN,2021-12-01 10:44:00,NaN,NaN,"[""9W remaining meds""]",NaN,NaN,NaN,Wednesday
2799,2021-12-01,1.02109e+09,102,NaN,SCM,1,2021-12-01 11:30:00,2021-12-01 11:42:00,NaN,NaN,2021-12-01 12:08:00,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday
2800,2021-12-01,1.02108e+09,113,NaN,SCM,1,2021-12-01 12:26:00,2021-12-01 12:38:00,NaN,NaN,2021-12-01 12:45:00,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday
2801,2021-12-01,1.02108e+09,113,NaN,SCM,1,2021-12-01 13:11:00,2021-12-01 13:17:00,NaN,NaN,2021-12-01 13:34:00,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday


## EDA

### Overall Time Taken (mins)

In [174]:
#merged_df["DateTime TTO Dispensed"] = pd.to_datetime(merged_df[merged_df['DateTime TTO Dispensed'] != 'NA']['DateTime TTO Dispensed'].astype(str))
#merged_df["DateTime TTO Counselled"]= pd.to_datetime(merged_df[merged_df['DateTime TTO Counselled'] != 'NA']['DateTime TTO Counselled'].astype(str))

In [175]:
#merged_df["TTO Last Timestamp"] = merged_df[['DateTime TTO Reached', 'DateTime TTO Counselled']].max(axis=1)
#merged_df["Last Timestamp"]  = merged_df['Last Timestamp'].dt.time

In [176]:
#columns = ['Time TTO Reached', 'Time TTO Counselled', 'Last Timestamp']

#for column in column:
#    merged_df[column]  = merged_df[column].dt.time

In [177]:
#merged_df["Time TTO Received"]  = pd.to_datetime(merged_df["Time TTO Received"] , infer_datetime_format=True, errors='coerce')
#merged_df["Time TTO Reached"]  = pd.to_datetime(merged_df["Time TTO Reached"] , infer_datetime_format=True, errors='coerce')
#merged_df["Time TTO Counselled"]  = pd.to_datetime(merged_df["Time TTO Counselled"] , infer_datetime_format=True, errors='coerce')

In [178]:
#merged_df["DateTime TTO Dispensed"]  = merged_df['DateTime TTO Dispensed'].dt.time
#merged_df["DateTime TTO Counselled"]  = merged_df['DateTime TTO Counselled'].dt.time

In [179]:
#merged_df["TTO Last Timestamp"]  = merged_df['TTO Last Timestamp'].dt.time

In [180]:
#merged_df.info()

In [181]:
#merged_df

In [182]:
tto_received = pd.to_datetime(merged_df['DateTime TTO Received'].astype(str)) 
tto_dispensed = pd.to_datetime(merged_df[merged_df['DateTime TTO Dispensed'] != 'NA']['DateTime TTO Dispensed'].astype(str))
#tto_counselled = pd.to_datetime(df2[df2['Time TTO Counselled'] != 'NA']['Time TTO Counselled'].astype(str))

merged_df['TAT'] = tto_dispensed.sub(tto_received).dt.total_seconds().div(60)

In [183]:
merged_df

,Date,Case Number,Room,Ward,TTO Type,No. of Drugs,DateTime TTO Received,DateTime TTO Checked,Checked by,Time to meet,DateTime TTO Dispensed,DateTime (Counselled),Counselled by,Remarks,Limo Service,Time Taken (Received to Checked),Time Taken (Received to Dispensed),Weekday,TAT
0,2021-04-01,1021023186,957,9W,SCM,2,2021-04-01 08:59:00,2021-04-01 09:04:00,LET,09:44:00,2021-04-01 09:29:00,09:35:00,LET,NaN,NaN,00:05:00,00:30:00,Thursday,30.0
1,2021-04-01,1021023191,509,5E,SCM,3,2021-04-01 09:05:00,2021-04-01 09:26:00,LET,09:50:00,2021-04-01 09:29:00,09:32:00,ELISE,NaN,NaN,00:21:00,00:24:00,Thursday,24.0
2,2021-04-01,1021023190,508,5E,SCM,4,2021-04-01 09:07:00,2021-04-01 09:19:00,LET,09:52:00,2021-04-01 09:32:00,10:17:00,ELISE,P,NaN,00:12:00,00:25:00,Thursday,25.0
3,2021-04-01,1021023367,932,9E,SCM,2,2021-04-01 09:11:00,2021-04-01 09:22:00,ELISE,09:56:00,2021-04-01 09:37:00,09:40:00,LET,NaN,NaN,00:11:00,00:26:00,Thursday,26.0
4,2021-04-01,1021023455,909,9E,SCM,5,2021-04-01 09:36:00,2021-04-01 10:02:00,ELISE,10:21:00,2021-04-01 10:21:00,10:25:00,ELISE,NaN,NaN,00:26:00,00:45:00,Thursday,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2798,2021-12-01,1.02109e+09,905,NaN,SCM,9,2021-12-01 09:39:00,2021-12-01 10:30:00,NaN,NaN,2021-12-01 10:44:00,NaN,NaN,"[""9W remaining meds""]",NaN,NaN,NaN,Wednesday,65.0
2799,2021-12-01,1.02109e+09,102,NaN,SCM,1,2021-12-01 11:30:00,2021-12-01 11:42:00,NaN,NaN,2021-12-01 12:08:00,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,38.0
2800,2021-12-01,1.02108e+09,113,NaN,SCM,1,2021-12-01 12:26:00,2021-12-01 12:38:00,NaN,NaN,2021-12-01 12:45:00,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,19.0
2801,2021-12-01,1.02108e+09,113,NaN,SCM,1,2021-12-01 13:11:00,2021-12-01 13:17:00,NaN,NaN,2021-12-01 13:34:00,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,23.0


### Meet KPI

In [184]:
def f(row):
    if row['TAT'] > 45:
        val = 'No'
    elif row['TAT'] > 0 and row['TAT'] <= 45:
        val = 'Yes'
    else:
        val = 'NA'
    return val

In [185]:
merged_df['Meet KPI'] = merged_df.apply(f, axis=1)
merged_df

,Date,Case Number,Room,Ward,TTO Type,No. of Drugs,DateTime TTO Received,DateTime TTO Checked,Checked by,Time to meet,DateTime TTO Dispensed,DateTime (Counselled),Counselled by,Remarks,Limo Service,Time Taken (Received to Checked),Time Taken (Received to Dispensed),Weekday,TAT,Meet KPI
0,2021-04-01,1021023186,957,9W,SCM,2,2021-04-01 08:59:00,2021-04-01 09:04:00,LET,09:44:00,2021-04-01 09:29:00,09:35:00,LET,NaN,NaN,00:05:00,00:30:00,Thursday,30.0,Yes
1,2021-04-01,1021023191,509,5E,SCM,3,2021-04-01 09:05:00,2021-04-01 09:26:00,LET,09:50:00,2021-04-01 09:29:00,09:32:00,ELISE,NaN,NaN,00:21:00,00:24:00,Thursday,24.0,Yes
2,2021-04-01,1021023190,508,5E,SCM,4,2021-04-01 09:07:00,2021-04-01 09:19:00,LET,09:52:00,2021-04-01 09:32:00,10:17:00,ELISE,P,NaN,00:12:00,00:25:00,Thursday,25.0,Yes
3,2021-04-01,1021023367,932,9E,SCM,2,2021-04-01 09:11:00,2021-04-01 09:22:00,ELISE,09:56:00,2021-04-01 09:37:00,09:40:00,LET,NaN,NaN,00:11:00,00:26:00,Thursday,26.0,Yes
4,2021-04-01,1021023455,909,9E,SCM,5,2021-04-01 09:36:00,2021-04-01 10:02:00,ELISE,10:21:00,2021-04-01 10:21:00,10:25:00,ELISE,NaN,NaN,00:26:00,00:45:00,Thursday,45.0,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2798,2021-12-01,1.02109e+09,905,NaN,SCM,9,2021-12-01 09:39:00,2021-12-01 10:30:00,NaN,NaN,2021-12-01 10:44:00,NaN,NaN,"[""9W remaining meds""]",NaN,NaN,NaN,Wednesday,65.0,No
2799,2021-12-01,1.02109e+09,102,NaN,SCM,1,2021-12-01 11:30:00,2021-12-01 11:42:00,NaN,NaN,2021-12-01 12:08:00,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,38.0,Yes
2800,2021-12-01,1.02108e+09,113,NaN,SCM,1,2021-12-01 12:26:00,2021-12-01 12:38:00,NaN,NaN,2021-12-01 12:45:00,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,19.0,Yes
2801,2021-12-01,1.02108e+09,113,NaN,SCM,1,2021-12-01 13:11:00,2021-12-01 13:17:00,NaN,NaN,2021-12-01 13:34:00,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,23.0,Yes


In [186]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2803 entries, 0 to 2802
Data columns (total 20 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Date                                2803 non-null   datetime64[ns]
 1   Case Number                         2803 non-null   object        
 2   Room                                2803 non-null   object        
 3   Ward                                491 non-null    object        
 4   TTO Type                            2794 non-null   object        
 5   No. of Drugs                        2803 non-null   object        
 6   DateTime TTO Received               2774 non-null   datetime64[ns]
 7   DateTime TTO Checked                2581 non-null   datetime64[ns]
 8   Checked by                          466 non-null    object        
 9   Time to meet                        491 non-null    object        
 10  DateTime TTO Dispensed  

### Office Hour

In [187]:
x = datetime.strptime('08:30:00', '%H:%M:%S').time()
x

datetime.time(8, 30)

In [188]:
#merged_df["DateTime TTO Received"]  = pd.to_datetime(merged_df["DateTime TTO Received"].astype(str))
#merged_df["DateTime TTO Dispensed"]  = pd.to_datetime(merged_df["DateTime TTO Dispensed"].astype(str))
#merged_df["TTO Last Timestamp"]  = pd.to_datetime(merged_df["TTO Last Timestamp"] , infer_datetime_format=True, errors='coerce')

In [189]:
#merged_df

In [190]:
#merged_df["DateTime TTO Received"]   = merged_df["DateTime TTO Received"].dt.time
#merged_df["DateTime TTO Dispensed"]   = merged_df["DateTime TTO Dispensed"].dt.time

In [191]:
merged_df['Office Hours'] = np.where(merged_df['DateTime TTO Received'].isnull() | merged_df['DateTime TTO Dispensed'].isnull() , 'NA',
                            np.where(  #(dfinal['Time TTO Received'] >= pd.to_datetime('08:30:00').time()) & 
                                        (merged_df['DateTime TTO Dispensed'].dt.time <= pd.to_datetime('20:00:00').time()),
                                        'Yes', 'No'))

merged_df

,Date,Case Number,Room,Ward,TTO Type,No. of Drugs,DateTime TTO Received,DateTime TTO Checked,Checked by,Time to meet,...,DateTime (Counselled),Counselled by,Remarks,Limo Service,Time Taken (Received to Checked),Time Taken (Received to Dispensed),Weekday,TAT,Meet KPI,Office Hours
0,2021-04-01,1021023186,957,9W,SCM,2,2021-04-01 08:59:00,2021-04-01 09:04:00,LET,09:44:00,...,09:35:00,LET,NaN,NaN,00:05:00,00:30:00,Thursday,30.0,Yes,Yes
1,2021-04-01,1021023191,509,5E,SCM,3,2021-04-01 09:05:00,2021-04-01 09:26:00,LET,09:50:00,...,09:32:00,ELISE,NaN,NaN,00:21:00,00:24:00,Thursday,24.0,Yes,Yes
2,2021-04-01,1021023190,508,5E,SCM,4,2021-04-01 09:07:00,2021-04-01 09:19:00,LET,09:52:00,...,10:17:00,ELISE,P,NaN,00:12:00,00:25:00,Thursday,25.0,Yes,Yes
3,2021-04-01,1021023367,932,9E,SCM,2,2021-04-01 09:11:00,2021-04-01 09:22:00,ELISE,09:56:00,...,09:40:00,LET,NaN,NaN,00:11:00,00:26:00,Thursday,26.0,Yes,Yes
4,2021-04-01,1021023455,909,9E,SCM,5,2021-04-01 09:36:00,2021-04-01 10:02:00,ELISE,10:21:00,...,10:25:00,ELISE,NaN,NaN,00:26:00,00:45:00,Thursday,45.0,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2798,2021-12-01,1.02109e+09,905,NaN,SCM,9,2021-12-01 09:39:00,2021-12-01 10:30:00,NaN,NaN,...,NaN,NaN,"[""9W remaining meds""]",NaN,NaN,NaN,Wednesday,65.0,No,Yes
2799,2021-12-01,1.02109e+09,102,NaN,SCM,1,2021-12-01 11:30:00,2021-12-01 11:42:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,38.0,Yes,Yes
2800,2021-12-01,1.02108e+09,113,NaN,SCM,1,2021-12-01 12:26:00,2021-12-01 12:38:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,19.0,Yes,Yes
2801,2021-12-01,1.02108e+09,113,NaN,SCM,1,2021-12-01 13:11:00,2021-12-01 13:17:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,23.0,Yes,Yes


In [192]:
merged_df['Time Taken (Checked to Dispensed)'] = (merged_df['DateTime TTO Dispensed'] - merged_df['DateTime TTO Checked'])

### Average time taken

In [193]:
#Get Duration Difference of Received to Checked
merged_df['Time Taken (Received to Checked)'] = (merged_df['DateTime TTO Checked'] - merged_df['DateTime TTO Received'])
#Get Duration Difference of Checked to Dispensed
merged_df['Time Taken (Checked to Dispensed)'] = (merged_df['DateTime TTO Dispensed'] - merged_df['DateTime TTO Checked'])

In [194]:
#Create new dataframe to get results of Average Time Taken for each (Received to Checked) & (Checked to Dispensed)
dfinal_avg_time = merged_df[['Date', 'Time Taken (Received to Checked)', 'Time Taken (Checked to Dispensed)']]
dfinal_avg_time.head()

,Date,Time Taken (Received to Checked),Time Taken (Checked to Dispensed)
0,2021-04-01,00:05:00,00:25:00
1,2021-04-01,00:21:00,00:03:00
2,2021-04-01,00:12:00,00:13:00
3,2021-04-01,00:11:00,00:15:00
4,2021-04-01,00:26:00,00:19:00


In [195]:
#Convert NaT to 00:00:00
dfinal_avg_time.loc[dfinal_avg_time['Time Taken (Received to Checked)'].isnull(), 'Time Taken (Received to Checked)'] = pd.to_timedelta(0)
dfinal_avg_time.loc[dfinal_avg_time['Time Taken (Checked to Dispensed)'].isnull(), 'Time Taken (Checked to Dispensed)'] = pd.to_timedelta(0)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [196]:
#Re-instate time format
#dfinal_avg_time['Time Taken (Received to Checked)'] = pd.to_datetime(dfinal_avg_time['Time Taken (Received to Checked)'], infer_datetime_format=True).dt.time
#dfinal_avg_time['Time Taken (Checked to Dispensed)'] = pd.to_datetime(dfinal_avg_time['Time Taken (Checked to Dispensed)'], infer_datetime_format=True).dt.time

#Get average time taken per day
dfinal_avg_time = dfinal_avg_time.groupby(pd.to_datetime(dfinal_avg_time['Date']).dt.date).mean(numeric_only=False)
dfinal_avg_time

,Date,Time Taken (Received to Checked),Time Taken (Checked to Dispensed)
Date,,,
2021-04-01,2021-04-01,00:14:40,00:14:32
2021-04-03,2021-04-03,00:08:00,00:04:00
2021-04-05,2021-04-05,00:16:40,00:19:40
2021-04-06,2021-04-06,00:13:34.285714,00:15:42.857142
2021-04-07,2021-04-07,00:13:25.714285,00:15:04.285714
...,...,...,...
2021-12-27,2021-12-27,00:21:07.500000,00:09:00
2021-12-28,2021-12-28,00:10:23.076923,-1 days +23:32:13.846153
2021-12-29,2021-12-29,00:26:34.736842,00:24:28.421052


In [197]:
#Remove miliseconds
dfinal_avg_time['Time Taken (Received to Checked)'] = dfinal_avg_time['Time Taken (Received to Checked)'].dt.floor('s')
dfinal_avg_time['Time Taken (Checked to Dispensed)'] = dfinal_avg_time['Time Taken (Checked to Dispensed)'].dt.floor('s')

dfinal_avg_time

,Date,Time Taken (Received to Checked),Time Taken (Checked to Dispensed)
Date,,,
2021-04-01,2021-04-01,00:14:40,00:14:32
2021-04-03,2021-04-03,00:08:00,00:04:00
2021-04-05,2021-04-05,00:16:40,00:19:40
2021-04-06,2021-04-06,00:13:34,00:15:42
2021-04-07,2021-04-07,00:13:25,00:15:04
...,...,...,...
2021-12-27,2021-12-27,00:21:07,00:09:00
2021-12-28,2021-12-28,00:10:23,-1 days +23:32:13
2021-12-29,2021-12-29,00:26:34,00:24:28


In [198]:
#Rename Columns
dfinal_avg_time = dfinal_avg_time.rename({'Time Taken (Received to Checked)' : 'Avg Time Taken / Day (Received to Checked)', 'Time Taken (Checked to Dispensed)' : 'Avg Time Taken / Day (Checked to Dispensed)'}, axis=1)

In [199]:
dfinal_avg_time = dfinal_avg_time.reset_index(drop=True)
dfinal_avg_time

,Date,Avg Time Taken / Day (Received to Checked),Avg Time Taken / Day (Checked to Dispensed)
0,2021-04-01,00:14:40,00:14:32
1,2021-04-03,00:08:00,00:04:00
2,2021-04-05,00:16:40,00:19:40
3,2021-04-06,00:13:34,00:15:42
4,2021-04-07,00:13:25,00:15:04
...,...,...,...
248,2021-12-27,00:21:07,00:09:00
249,2021-12-28,00:10:23,-1 days +23:32:13
250,2021-12-29,00:26:34,00:24:28
251,2021-12-30,00:14:12,00:18:48


In [200]:
#Merge the results from grouby for Avg Time Taken
merged_df = merged_df.merge(dfinal_avg_time, on='Date', how='left')
merged_df

,Date,Case Number,Room,Ward,TTO Type,No. of Drugs,DateTime TTO Received,DateTime TTO Checked,Checked by,Time to meet,...,Limo Service,Time Taken (Received to Checked),Time Taken (Received to Dispensed),Weekday,TAT,Meet KPI,Office Hours,Time Taken (Checked to Dispensed),Avg Time Taken / Day (Received to Checked),Avg Time Taken / Day (Checked to Dispensed)
0,2021-04-01,1021023186,957,9W,SCM,2,2021-04-01 08:59:00,2021-04-01 09:04:00,LET,09:44:00,...,NaN,00:05:00,00:30:00,Thursday,30.0,Yes,Yes,00:25:00,00:14:40,00:14:32
1,2021-04-01,1021023191,509,5E,SCM,3,2021-04-01 09:05:00,2021-04-01 09:26:00,LET,09:50:00,...,NaN,00:21:00,00:24:00,Thursday,24.0,Yes,Yes,00:03:00,00:14:40,00:14:32
2,2021-04-01,1021023190,508,5E,SCM,4,2021-04-01 09:07:00,2021-04-01 09:19:00,LET,09:52:00,...,NaN,00:12:00,00:25:00,Thursday,25.0,Yes,Yes,00:13:00,00:14:40,00:14:32
3,2021-04-01,1021023367,932,9E,SCM,2,2021-04-01 09:11:00,2021-04-01 09:22:00,ELISE,09:56:00,...,NaN,00:11:00,00:26:00,Thursday,26.0,Yes,Yes,00:15:00,00:14:40,00:14:32
4,2021-04-01,1021023455,909,9E,SCM,5,2021-04-01 09:36:00,2021-04-01 10:02:00,ELISE,10:21:00,...,NaN,00:26:00,00:45:00,Thursday,45.0,Yes,Yes,00:19:00,00:14:40,00:14:32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2798,2021-12-01,1.02109e+09,905,NaN,SCM,9,2021-12-01 09:39:00,2021-12-01 10:30:00,NaN,NaN,...,NaN,00:51:00,NaN,Wednesday,65.0,No,Yes,00:14:00,00:11:25,00:16:30
2799,2021-12-01,1.02109e+09,102,NaN,SCM,1,2021-12-01 11:30:00,2021-12-01 11:42:00,NaN,NaN,...,NaN,00:12:00,NaN,Wednesday,38.0,Yes,Yes,00:26:00,00:11:25,00:16:30
2800,2021-12-01,1.02108e+09,113,NaN,SCM,1,2021-12-01 12:26:00,2021-12-01 12:38:00,NaN,NaN,...,NaN,00:12:00,NaN,Wednesday,19.0,Yes,Yes,00:07:00,00:11:25,00:16:30
2801,2021-12-01,1.02108e+09,113,NaN,SCM,1,2021-12-01 13:11:00,2021-12-01 13:17:00,NaN,NaN,...,NaN,00:06:00,NaN,Wednesday,23.0,Yes,Yes,00:17:00,00:11:25,00:16:30


In [201]:
# drop Time Taken (Received to Dispensed) since it is the same as TAT
merged_df.drop(['Time Taken (Received to Dispensed)'], axis=1, inplace=True)

In [202]:
merged_df['TTO Type'] = merged_df['TTO Type'].replace('HW TTO', 'Handwritten')

### Average Prescriptions Filled Per Month

In [203]:
#df['Date'] = pd.to_datetime(df['Date'])
#df.groupby(df['Date'].dt.strftime('%B'))['No. of Drugs'].sum().sort_values()

In [204]:
#df.groupby(df['Date'].dt.strftime('%B'))['No. of Items'].mean().sort_values()

### Ward

Match Room No. to the corresponding Ward No.

In [205]:
df_ward = pd.read_excel("../../Data/Pharmacy Dept/GEH TTO Data/GEH Wards.xlsx", sheet_name=0)

df_ward

,Room,Ward
0,1,ICU
1,2,ICU
2,3,ICU
3,4,ICU
4,5,ICU
...,...,...
311,NICU 5,NICU
312,NICU 5,NICU
313,NICU 6,NICU
314,NICU 7,NICU


In [206]:
df_ward.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316 entries, 0 to 315
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Room    316 non-null    object
 1   Ward    316 non-null    object
dtypes: object(2)
memory usage: 5.1+ KB


In [207]:
df_ward['Room'] = df_ward['Room'].astype(str)
merged_df['Room'] = merged_df['Room'].astype(str)

In [208]:
dfinal = pd.merge(merged_df, df_ward, how='left', on=['Room'])

dfinal.head()

,Date,Case Number,Room,Ward_x,TTO Type,No. of Drugs,DateTime TTO Received,DateTime TTO Checked,Checked by,Time to meet,...,Limo Service,Time Taken (Received to Checked),Weekday,TAT,Meet KPI,Office Hours,Time Taken (Checked to Dispensed),Avg Time Taken / Day (Received to Checked),Avg Time Taken / Day (Checked to Dispensed),Ward_y
0,2021-04-01,1021023186,957,9W,SCM,2,2021-04-01 08:59:00,2021-04-01 09:04:00,LET,09:44:00,...,NaN,00:05:00,Thursday,30.0,Yes,Yes,00:25:00,00:14:40,00:14:32,9W
1,2021-04-01,1021023191,509,5E,SCM,3,2021-04-01 09:05:00,2021-04-01 09:26:00,LET,09:50:00,...,NaN,00:21:00,Thursday,24.0,Yes,Yes,00:03:00,00:14:40,00:14:32,5E
2,2021-04-01,1021023190,508,5E,SCM,4,2021-04-01 09:07:00,2021-04-01 09:19:00,LET,09:52:00,...,NaN,00:12:00,Thursday,25.0,Yes,Yes,00:13:00,00:14:40,00:14:32,5E
3,2021-04-01,1021023367,932,9E,SCM,2,2021-04-01 09:11:00,2021-04-01 09:22:00,ELISE,09:56:00,...,NaN,00:11:00,Thursday,26.0,Yes,Yes,00:15:00,00:14:40,00:14:32,9E
4,2021-04-01,1021023455,909,9E,SCM,5,2021-04-01 09:36:00,2021-04-01 10:02:00,ELISE,10:21:00,...,NaN,00:26:00,Thursday,45.0,Yes,Yes,00:19:00,00:14:40,00:14:32,9E


In [209]:
dfinal[dfinal['Ward_y'].isnull()]

,Date,Case Number,Room,Ward_x,TTO Type,No. of Drugs,DateTime TTO Received,DateTime TTO Checked,Checked by,Time to meet,...,Limo Service,Time Taken (Received to Checked),Weekday,TAT,Meet KPI,Office Hours,Time Taken (Checked to Dispensed),Avg Time Taken / Day (Received to Checked),Avg Time Taken / Day (Checked to Dispensed),Ward_y
261,2021-04-29,1021030734,6W03,6W Nursery,SCM,1,2021-04-29 10:00:00,2021-04-29 10:34:00,SHA,10:45:00,...,NaN,00:34:00,Thursday,NaN,NA,NA,NaT,00:27:32,-1 days +23:34:00,NaN
621,2021-06-14,1.02104e+09,endo,NaN,Handwritten,2,2021-06-14 15:29:00,NaT,NaN,NaN,...,NaN,NaT,Monday,NaN,NA,NA,NaT,00:21:28,00:21:22,NaN
625,2021-06-15,1.02104e+09,OT,NaN,SCM,1,2021-06-15 08:55:00,NaT,NaN,NaN,...,NaN,NaT,Tuesday,NaN,NA,NA,NaT,00:14:21,00:11:56,NaN
717,2021-06-22,1.02104e+09,737,NaN,SCM,1,2021-06-22 10:10:00,2021-06-22 10:26:00,NaN,NaN,...,NaN,00:16:00,Tuesday,105.000000,No,Yes,01:29:00,00:41:02,00:23:31,NaN
721,2021-06-22,1.02104e+09,737,NaN,SCM,2,2021-06-22 11:15:00,2021-06-22 11:35:00,NaN,NaN,...,NaN,00:20:00,Tuesday,41.000000,Yes,Yes,00:21:00,00:41:02,00:23:31,NaN
802,2021-06-29,1.02105e+09,icu 05,NaN,Handwritten,1,2021-06-29 14:21:00,NaT,NaN,NaN,...,NaN,NaT,Tuesday,NaN,NA,NA,NaT,00:06:08,00:28:43,NaN
887,2021-07-07,1.02105e+09,737,NaN,SCM,7,2021-07-07 09:30:00,2021-07-07 09:46:00,NaN,NaN,...,NaN,00:16:00,Wednesday,49.000000,No,Yes,00:33:00,00:30:04,00:23:12,NaN
913,2021-07-09,1.02105e+09,406,NaN,SCM,6,2021-07-09 09:10:00,NaT,NaN,NaN,...,NaN,NaT,Friday,NaN,NA,NA,NaT,01:05:33,00:22:20,NaN
1020,2021-07-17,1.02105e+09,737,NaN,SCM,3,2021-07-17 09:07:00,2021-07-17 09:33:00,NaN,NaN,...,NaN,00:26:00,Saturday,62.000000,No,Yes,00:36:00,00:17:14,00:26:00,NaN
1034,2021-07-18,1.02105e+09,737,NaN,SCM,5,2021-07-18 08:55:00,2021-07-18 09:02:00,NaN,NaN,...,NaN,00:07:00,Sunday,NaN,NA,NA,NaT,00:09:24,00:02:42,NaN


#### Fix invalid Room

In [210]:
#fixing values in Room that are actually referring to the Ward, and match its corresponding in the Ward column
dfinal.loc[(dfinal['Room'].str.contains('endo', case=False, na=False)), 'Ward_y'] = 'ENDO'
dfinal.loc[(dfinal['Room'].str.contains('6W', case=False, na=False)), 'Ward_y'] = '6W'
#dfinal.loc[(dfinal['Room'].str.contains('icu', case=False, na=False)), 'Ward_y'] = 'ICU'

In [211]:
dfinal.drop(['Ward_x'], axis=1, inplace=True)
dfinal.rename({'Ward_y': 'Ward'}, axis=1, inplace=True)

In [215]:
dfinal

,Date,Case Number,Room,TTO Type,No. of Drugs,DateTime TTO Received,DateTime TTO Checked,Checked by,Time to meet,DateTime TTO Dispensed,...,Limo Service,Time Taken (Received to Checked),Weekday,TAT,Meet KPI,Office Hours,Time Taken (Checked to Dispensed),Avg Time Taken / Day (Received to Checked),Avg Time Taken / Day (Checked to Dispensed),Ward
0,2021-04-01,1021023186,957,SCM,2,2021-04-01 08:59:00,2021-04-01 09:04:00,LET,09:44:00,2021-04-01 09:29:00,...,NaN,00:05:00,Thursday,30.0,Yes,Yes,00:25:00,00:14:40,00:14:32,9W
1,2021-04-01,1021023191,509,SCM,3,2021-04-01 09:05:00,2021-04-01 09:26:00,LET,09:50:00,2021-04-01 09:29:00,...,NaN,00:21:00,Thursday,24.0,Yes,Yes,00:03:00,00:14:40,00:14:32,5E
2,2021-04-01,1021023190,508,SCM,4,2021-04-01 09:07:00,2021-04-01 09:19:00,LET,09:52:00,2021-04-01 09:32:00,...,NaN,00:12:00,Thursday,25.0,Yes,Yes,00:13:00,00:14:40,00:14:32,5E
3,2021-04-01,1021023367,932,SCM,2,2021-04-01 09:11:00,2021-04-01 09:22:00,ELISE,09:56:00,2021-04-01 09:37:00,...,NaN,00:11:00,Thursday,26.0,Yes,Yes,00:15:00,00:14:40,00:14:32,9E
4,2021-04-01,1021023455,909,SCM,5,2021-04-01 09:36:00,2021-04-01 10:02:00,ELISE,10:21:00,2021-04-01 10:21:00,...,NaN,00:26:00,Thursday,45.0,Yes,Yes,00:19:00,00:14:40,00:14:32,9E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2798,2021-12-01,1.02109e+09,905,SCM,9,2021-12-01 09:39:00,2021-12-01 10:30:00,NaN,NaN,2021-12-01 10:44:00,...,NaN,00:51:00,Wednesday,65.0,No,Yes,00:14:00,00:11:25,00:16:30,9E
2799,2021-12-01,1.02109e+09,102,SCM,1,2021-12-01 11:30:00,2021-12-01 11:42:00,NaN,NaN,2021-12-01 12:08:00,...,NaN,00:12:00,Wednesday,38.0,Yes,Yes,00:26:00,00:11:25,00:16:30,10W
2800,2021-12-01,1.02108e+09,113,SCM,1,2021-12-01 12:26:00,2021-12-01 12:38:00,NaN,NaN,2021-12-01 12:45:00,...,NaN,00:12:00,Wednesday,19.0,Yes,Yes,00:07:00,00:11:25,00:16:30,10W
2801,2021-12-01,1.02108e+09,113,SCM,1,2021-12-01 13:11:00,2021-12-01 13:17:00,NaN,NaN,2021-12-01 13:34:00,...,NaN,00:06:00,Wednesday,23.0,Yes,Yes,00:17:00,00:11:25,00:16:30,10W


## Export Final CLEANED to csv 

In [216]:
dfinal.to_csv("../../Data/Pharmacy Dept/Data Cleaning/GEH_Data_Cleaned.csv", index = None, header=True)